Nhóm 18
- Đỗ Tấn Lực - 23520903
- Lê Quang Long - -23520878

#### Import các thư viện cần thiết cho xử lý dữ liệu, xây dựng mô hình, và Spark

- pandas, numpy: Xử lý dữ liệu.
- torch, torch.nn, torch.optim: Xây dựng và huấn luyện mô hình deep learning.
- sklearn: Tiền xử lý, chia tập, đánh giá mô hình.
- re: Xử lý chuỗi, regex.
- kagglehub: Tải dataset từ Kaggle.
- pyspark: Xử lý dữ liệu lớn với Spark, định nghĩa UDF.
- transformers: Sử dụng mô hình BERT (PhoBERT).

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from collections import Counter
import re
import kagglehub
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, ArrayType, StringType, IntegerType
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.functions import predict_batch_udf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from transformers import AutoTokenizer, AutoModel

#### Tải dataset ABSA tiếng Việt từ Kaggle

- Sử dụng kagglehub để tải về bộ dữ liệu.
- Đọc các file train, test, val bằng pandas.

In [ ]:
path = kagglehub.dataset_download("cthng123/absa-vietnamese")

train = pd.read_csv(f"{path}/train_data.csv")
test = pd.read_csv(f"{path}/test_data.csv")
val = pd.read_csv(f"{path}/val_data.csv")

## BERT + Predict batch UDF

- Sử dụng mô hình BERT (PhoBERT) để dự đoán song song trên Spark DataFrame.
- Trả về dự đoán cho từng review dưới dạng chuỗi.
- Đánh giá lại độ chính xác trên Spark DataFrame.

#### Cài đặt thư viện transformers

- Sử dụng pip để cài đặt phiên bản `transformers==4.40.2`.

In [ ]:
!pip install transformers==4.40.2 --quiet

#### Tải tokenizer và mô hình PhoBERT

- Tải `AutoTokenizer` và `AutoModel` từ thư viện `transformers`.
- Khởi tạo tokenizer cho mô hình `vinai/phobert-base`.

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Định nghĩa mô hình ABSA dựa trên BERT và khởi tạo

- Định nghĩa lớp `BERT_ABSA` kế thừa từ `nn.Module`.
- Mô hình bao gồm lớp BERT (`vinai/phobert-base`), dropout, và lớp tuyến tính để dự đoán phân loại cho từng khía cạnh.
- Khởi tạo mô hình, hàm mất mát `CrossEntropyLoss`, và optimizer `Adam`.
- Chuyển mô hình sang thiết bị (CPU/GPU) được định nghĩa bởi biến `DEVICE`.

In [ ]:
class BERT_ABSA(nn.Module):
    def __init__(self, num_aspects=len(ASPECTS), num_classes=NUM_CLASSES):
        super().__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_aspects * num_classes)
        self.num_classes = num_classes
        self.num_aspects = num_aspects

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        drop = self.dropout(pooled_output)
        out = self.fc(drop)
        return out.view(-1, self.num_aspects, self.num_classes)


model_bert = BERT_ABSA().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_bert.parameters(), lr=0.001)

#### Token hóa dữ liệu và chuẩn bị Tensor

- Định nghĩa hàm `tokenize` để chuyển đổi văn bản thành input_ids và attention_mask sử dụng tokenizer của PhoBERT.
- Áp dụng hàm `tokenize` cho cột 'Review' trong các tập train, test, và val.
- Nối các tensor input_ids và attention_mask lại với nhau cho từng tập dữ liệu.

In [ ]:
def tokenize(text):
    return tokenizer.encode_plus(text, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')

train_bert = [tokenize(text) for text in train['Review']]
test_bert = [tokenize(text) for text in test['Review']]
val_bert = [tokenize(text) for text in val['Review']]

train_input_ids = torch.cat([e['input_ids'] for e in train_bert])
train_attention_mask = torch.cat([e['attention_mask'] for e in train_bert])
val_input_ids = torch.cat([e['input_ids'] for e in val_bert])
val_attention_mask = torch.cat([e['attention_mask'] for e in val_bert])
test_input_ids = torch.cat([e['input_ids'] for e in test_bert])
test_attention_mask = torch.cat([e['attention_mask'] for e in test_bert])


#### Định nghĩa Dataset, DataLoader

- Định nghĩa lớp `ABSADataset_Bert` kế thừa từ `Dataset` để xử lý dữ liệu cho PyTorch.
- Tạo instance của Dataset cho tập train, test, và val.
- Tạo DataLoader cho từng tập dữ liệu để xử lý theo batch.


In [ ]:
class ABSADataset_Bert(Dataset):
      def __init__(self, input_ids, attention_masks, labels):
            self.input_ids = input_ids
            self.attention_masks = attention_masks
            self.labels = torch.tensor(labels, dtype=torch.long)

      def __getitem__(self, idx):
            return self.input_ids[idx], self.attention_masks[idx], self.labels[idx]

      def __len__(self):
            return len(self.input_ids)


train_data = ABSADataset_Bert(train_input_ids, train_attention_mask, train_labels)
test_data = ABSADataset_Bert(test_input_ids, test_attention_mask, test_labels)
val_data = ABSADataset_Bert(val_input_ids, val_attention_mask, val_labels)

train_load = DataLoader(train_data, batch_size=32, shuffle=True)
test_load = DataLoader(test_data, batch_size=32)
val_load = DataLoader(val_data, batch_size=32)

Epoch 1/1, Loss: 5.652547785730073
Epoch 2, Val Loss: 0.0, Val Accuracy: 0.7489316239316239


#### Huấn luyện và đánh giá mô hình BERT

- Thực hiện vòng lặp huấn luyện mô hình:
    - Đặt mô hình ở chế độ train (`model_bert.train()`).
    - Tính tổng mất mát qua từng batch.
    - Thực hiện backpropagation và cập nhật trọng số.
    - In ra mất mát sau mỗi epoch.
- Đánh giá mô hình trên tập validation sau khi huấn luyện:
    - Đặt mô hình ở chế độ eval (`model_bert.eval()`).
    - Dự đoán trên tập validation.
    - Tính toán độ chính xác trên tập validation.
    - In ra độ chính xác.
- Lưu trạng thái của mô hình đã huấn luyện vào file 'bert_absa_model.pth'.

In [ ]:
epoch = 1
for e in range(epoch):
    model_bert.train()
    total_loss = 0
    for input_ids, attention_mask, labels in train_load:
        input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        output = model_bert(input_ids, attention_mask)
        loss = sum(criterion(output[:, i, :], labels[:, i]) for i in range(len(ASPECTS)))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {e+1}/{epoch}, Loss: {total_loss/len(train_load)}")

# EVAL
model_bert.eval()
val_pred = []
val_true = []
val_loss = 0
with torch.no_grad():
    for input_ids, attention_mask, labels in val_load:
        input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
        outputs = model_bert(input_ids, attention_mask)
        val_pred.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        val_true.extend(labels.cpu().numpy())
val_loss /= len(val_load)
val_acc = accuracy_score(np.array(val_true).flatten(), np.array(val_pred).flatten())
print(f'Epoch {epoch+1}, Val Loss: {val_loss}, Val Accuracy: {val_acc}')

torch.save(model_bert.state_dict(), 'bert_absa_model.pth')

#### Đánh giá mô hình trên tập test

- Đặt mô hình ở chế độ eval (`model_bert.eval()`).
- Thực hiện dự đoán trên tập test.
- So sánh kết quả dự đoán với nhãn thực tế.
- Tính toán độ chính xác cuối cùng trên tập test và in ra.

In [ ]:
model_bert.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_load:
        input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
        outputs = model_bert(input_ids, attention_mask)
        test_preds.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        test_true.extend(labels.cpu().numpy())
test_acc = accuracy_score(np.array(test_true).flatten(), np.array(test_preds).flatten())
print('Final Test Accuracy:', test_acc)

Final Test Accuracy: 0.753792735042735


#### Dự đoán song song với Spark predict_batch_udf (BERT)

- Sử dụng Spark để dự đoán song song theo batch trên tập test với mô hình đã huấn luyện.
- Trả về dự đoán cho từng review dưới dạng mảng số.
- Đánh giá lại độ chính xác trên Spark DataFrame.

In [ ]:
spark = SparkSession.builder.appName('ABSA_PredictBatchUDFBERT').getOrCreate()
spark_test_df = spark.createDataFrame(test[['Review']])

def predict_batch_bert():

    def predict_batch(reviews):
        load_model = BERT_ABSA().to('cpu')
        load_model.load_state_dict(torch.load('bert_absa_model.pth'))
        load_model.eval()

        tokens = [tokenize(t) for t in reviews]
        input_ids = torch.cat([t['input_ids'] for t in tokens])
        attention_mask = torch.cat([t['attention_mask'] for t in tokens])

        with torch.no_grad():
            output = load_model(input_ids, attention_mask)
            predict = torch.argmax(output, dim=2).numpy()
        return predict

    return predict_batch

absa_batch_udf_bert = predict_batch_udf(
    predict_batch_bert,
    return_type=ArrayType(IntegerType()),
    batch_size=32,
    ## input_tensor_shapes=[[1]]
)

# Apply UDF
result_df_udf = spark_test_df.withColumn('predictions', absa_batch_udf_bert(spark_test_df['Review']))
result_df_udf.show(truncate=False)

preds_df = spark.createDataFrame(pd.DataFrame({'prediction': np.array(test_preds).flatten(), 'label': np.array(test_true).flatten()}))
preds_df = preds_df.withColumn('prediction', preds_df['prediction'].cast(DoubleType()))
preds_df = preds_df.withColumn('label', preds_df['label'].cast(DoubleType()))

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(preds_df)
print('Spark Test Accuracy:', accuracy)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|Review                                                                                                                                                                                                                                               |predictions             |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|Giày hơi có mùi nồng, lưu ý đôi LA không phải đế xám nên mng cân nhắc kĩ nhé ạ nhma với giá tiền này thì là oke                                                                     